In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [14]:
# Wczytanie danych
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv"
column_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']

data = pd.read_csv(url, names=column_names)

# Podział danych na cechy i etykiety
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Podział danych na zestawy treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizacja danych
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [15]:
# Tworzenie modelu
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Jedna jednostka wyjściowa dla regresji
])

# Kompilacja modelu
model.compile(optimizer='adam', loss='mean_squared_error')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Callback do zapisu wag po każdej epoce
class WeightsSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        weights = model.get_weights()
        # Save each weight array separately
        for i, weight in enumerate(weights):
            np.save(f"weights_epoch_{epoch+1}_layer_{i}.npy", weight)

weights_saver = WeightsSaver()

In [16]:
# Trenowanie modelu z callbackiem
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[weights_saver])


Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.5172 - val_loss: 0.2727
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2704 - val_loss: 0.1884
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1832 - val_loss: 0.1787
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1808 - val_loss: 0.1763
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1547 - val_loss: 0.1780
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1617 - val_loss: 0.1759
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1682 - val_loss: 0.1747
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1331 - val_loss: 0.1739
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1487 - val_loss: 0.1744
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1217 - val_loss: 0.1713
Epoch 11/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1321 - val_loss: 0.1699
Epoch 12/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1334 - val_l

In [17]:
# Ocena modelu na danych testowych
test_loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2181 
Test Loss: 0.21175380051136017


In [18]:
# Przykładowe wczytanie wag z pliku po 10. epoce
loaded_weights = []
for i in range(6): # Assuming there are 6 weight arrays (adjust if necessary)
  loaded_weights.append(np.load(f"weights_epoch_10_layer_{i}.npy", allow_pickle=True))
model.set_weights(loaded_weights)
